In [ ]:
!pip install transformers torch sentencepiece nltk

In [ ]:
!pip install yake

In [ ]:
import yake

def extract_keyphrases(text, top_k=3):
    kw_extractor = yake.KeywordExtractor(
    n=2,
    dedupLim=0.7,
    windowsSize=3,
    )

    keywords = kw_extractor.extract_keywords(text)
    keyphrases = []
    current_phrase = []
    for keyword, score in keywords:
        keyphrases.append(keyword)

    return keyphrases[:top_k]

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer2 = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model2 = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2020: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [ ]:
def generate_question(context, answer):
    input_text = f"generate question: {context} answer: {answer}"
    inputs = tokenizer2(input_text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model2.generate(**inputs, max_new_tokens=50)
    question = tokenizer2.decode(outputs[0], skip_special_tokens=True)
    return question

In [ ]:
import nltk
from nltk.corpus import wordnet

nltk.download('wordnet')

def get_distractors(word, num=3):
    synsets = wordnet.synsets(word)
    distractors = []
    for syn in synsets:
        for lemma in syn.lemmas():
            if lemma.name() != word and lemma.name() not in distractors:
                distractors.append(lemma.name())
                if len(distractors) >= num:
                    return distractors
    return distractors

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import random

def generate_mcq(text, num_questions=3):
    keyphrases = extract_keyphrases(text, top_k=num_questions)
    mcqs = []
    for answer in keyphrases:
        question = generate_question(text, answer)

        mcq = {
            "question": question,
            "canswer": answer
        }
        mcqs.append(mcq)
    return mcqs

In [ ]:
text = "Photosynthesis converts sunlight into chemical energy."
mcqs = generate_mcq(text)
for i in mcqs:
  print(i)

{'question': 'question: What does photosynthesis do to sunlight?', 'correct_answer': 'Photosynthesis converts'}
{'question': 'question: What does photosynthesis convert sunlight into?', 'correct_answer': 'chemical energy'}
{'question': 'question: What does photosynthesis do?', 'correct_answer': 'converts sunlight'}
